# Marine Heat Wave analysis
The purpose of this notebook is to take in SST data, along with previously computed climatology and threshold, process these and output all the MHW events that occur in the data

First, we import the data and required modules. 

In [4]:
import xarray as xr
import pandas as pd
import numpy as np
import dask as da
import scipy.ndimage as ndimage
#import dask_image.ndmeasure <-- Would make computation faster - but not finished yet
from dask.distributed import LocalCluster, Client
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
cluster = LocalCluster(processes=False, local_dir= "/g/data1a/e14/rm6294/dask-workers")
client = Client(cluster)
client

/g/data1/e14/rm6294/miniconda3/envs/now/lib/python3.6/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: inproc://10.0.64.22/6973/1 Dashboard: http://localhost:35021/status,Cluster Workers: 1 Cores: 8 Memory: 33.67 GB


In [7]:
# Opens Climatology and MHW SST threshold files, that have been created before.
# Note: p stands for past, f for future
localDir = "/g/data1a/e14/rm6294/NOWMHW/"
pThresh = xr.open_dataarray(localDir + 'pastThreshRep.nc')
pClim = xr.open_dataarray(localDir + 'pastClimRep.nc')
fThresh = xr.open_dataarray(localDir + 'futureThreshRep.nc')
fClim = xr.open_dataarray(localDir + 'futureClimRep.nc')

In [8]:
# Opens NOW SST data, from which we need the SST.
sst_dir = '/g/data1a/e14/gs9353/NOW_OUTPUTS/'
sstFut = xr.open_zarr(sst_dir + 'now_cordex24_BMJv2_BILAP_ALL/cordex24-BMJv2_BILAP_ALL_1d_1990_2008_grid_T_2D.zarr')['tos'].sel(simulation = 'Future') #tos means sst
sstPast = xr.open_zarr(sst_dir + 'now_cordex24_BMJv2_BILAP_old/cordex24-BMJv2_BILAP_1d_1990_2008_grid_T_2D.zarr')['tos'].sel(simulation = 'Present')

In [35]:
spatial_chunk = 50

## Looking for MHW
The method we will be using to identify MHW is to:
* Create a mask on top of the data, which has a binary True/False representing whether the sst at the location is above the corressponding threshold. 
    * Using the mask, try and find consecutive days where the threshold is crossed, as these represent the MHW.
* Once we have all the SSTs that cross the threshold, we will the apply the duration requirement, and look for SSTs that cross the threshold for 5 consecutive days

### Creating a Mask (Crossing the threshold)
We are looking for SST to cross the threshold. We will do this by:
* Subtracting the background climatology from both the threshold and sst, giving us theshold anomaly (thresha) and SST anomaly (ssta).
* Dividing ssta by thresha.
* Saving the severity data, so we can simply call it for use in the animation section.
This process will not only give us a matrix of threshold crosses (as ssta > thresha at this points, ssta/thresa > 1), but also tell us about the severity of the MHW, as temperatures 2 thresholds away from the background will be represented with a two in the matrix and so on.

##### Note: Current looking at Future SSTs compared to a past Clim.

In [9]:
# Finding thresha and ssta
thresha = pThresh.chunk({'x':spatial_chunk, 'y':spatial_chunk, 'time_counter':365}) - pClim.chunk({'x':spatial_chunk, 'y':spatial_chunk, 'time_counter':365})
ssta = sstFut.chunk({'x':spatial_chunk, 'y':spatial_chunk, 'time_counter':365}) - pClim.chunk({'x':spatial_chunk, 'y':spatial_chunk, 'time_counter':365})

In [13]:
# Finding potential MHW events
mhws_preDur = xr.where(ssta > thresha, ssta/thresha, 0)
# The next line removes any points that were made very large due to epsilon error in the ssta/thresha div
mhws_preDur = xr.where(mhws_preDur > 10, 0, mhws_preDur)
# Below ensures that longitude coordinates go between 0 and 360. Makes plotting easier.
mhws_preDur['nav_lon'] = mhws_preDur['nav_lon'] % 360 

## Checking for duration
We now have all the SSTs that cross our threshold. This section will aim to find the events that meet the 5 consecutive days with at most a 2 day gap criterion. To do this we will:
* For each the time series at each grid point, track how long each MHW occurs for.
* Eliminate those that are less than 5 days long 

In [33]:
def duration(severity, join_gaps = True):
    # Finding the dates considered in severity & converting it to nanoseconds
    mhw_dates = pd.to_numeric(severity['time_counter'])
    # Creates mhw_dur object which has a time series (in nanoseconds) at each grid point
    mhw_dur = xr.broadcast(severity, mhw_dates)[1]
    mhw_dur.data = da.array.from_array(mhw_dur.data, chunks = (chunk_size, spatial_chunk, spatial_chunk))
    
    lbl = da.array.map_blocks(remove_false_events, mhw_dur.data, severity.data, dtype = 'float')
    
    return xr.where(lbl > 0, severity, 0)

In [15]:
def day2ns(days):
    # Converts days to nanoseconds. Days -> Hours -> Seconds -> nanoseconds 
    return int(days * 24 * 3600 * 1e9)

In [16]:
def remove_false_events(mhw_dur, severity, minDur = 5):
    struc = np.zeros((3,3,3))
    # Looking for consecutive SST threshold crosses for only the time dimension:
    struc[:,1,1] = 1
    labeled_array, num_features = ndimage.label(severity, structure = struc)   
    indexs = da.array.arange(1, num_features + 1)
   
    # Calculates the max and min time points for all labeled points, i.e. the beggining and end dates for SST events 
    maxMHW = ndimage.maximum(mhw_dur, labels=labeled_array, index=indexs)
    minMHW = ndimage.minimum(mhw_dur, labels=labeled_array, index=indexs)

    # Calculates the length of all potential MHW events.
    dur = maxMHW - minMHW + day2ns(1) 

    # converting minDur (default = 5) days to nanoseconds
    minDur_ns = day2ns(minDur) 

    # Finds the event numbers of MHWs with duration < minDur_ns.
    # Adding 1 so that event number matches with value in labeled_array. 
    failed_ev_num = da.array.where(dur < minDur_ns)[0]  + 1 
    failed_ev_num = failed_ev_num.compute()

    # objs has the location of each labeled event 
    # i.e objs[0] is the location of the event 1, objs[10] location of event 11 and so on.
    objs = ndimage.find_objects(labeled_array.astype('int'))
    
    # failed_objs has the location for all the failed events in labeled_array 
    failed_objs = [objs[i - 1] for i in failed_ev_num]
    # Removes all the places where we had a failed MHW
    for loc in failed_objs:
        labeled_array[loc] = 0

    return labeled_array

In [17]:
chunk_size = 431 # <- Chunk size needs to be the same as the max spatial dimension length 
mhws_preDur = mhws_preDur.chunk({'time_counter' : chunk_size})

In [36]:
mhw = duration(mhws_preDur)

In [37]:
mhw = mhw.compute()

/g/data1/e14/rm6294/miniconda3/envs/now/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater
  return func(*args2)
/g/data1/e14/rm6294/miniconda3/envs/now/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater
  return func(*args2)
/g/data1/e14/rm6294/miniconda3/envs/now/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater
  return func(*args2)
/g/data1/e14/rm6294/miniconda3/envs/now/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater
  return func(*args2)
/g/data1/e14/rm6294/miniconda3/envs/now/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater
  return func(*args2)
/g/data1/e14/rm6294/miniconda3/envs/now/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater
  return func(*args2)
/g/data1/e14/rm6294/miniconda3/env

In [ ]:
mhw.chunk({'x' : spatial_chunk, 'y' : spatial_chunk, 'time_counter' : 365}).to_netcdf('/g/data1a/e14/rm6294/NOWMHW/severityMHWNOW_futSSTPastClim.nc')